In [7]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from plantcv import plantcv as pcv
import pandas as pd

ImportError: cannot import name 'experimental_functions_run_eagerly' from 'tensorflow.python.eager.def_function' (c:\Users\saaha\OneDrive\Documents\GitHub\BiomassPredictonAI\venv\lib\site-packages\tensorflow\python\eager\def_function.py)

In [4]:
print(tf.__version__)
print(pd.__version__)

2.11.0
1.5.3


In [10]:
# How many parameters are we inputing into our model
num_parameters = 3

In [1]:
# See if there are any GPUs that can be used to run our code
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

NameError: name 'tf' is not defined

In [6]:
# created callback to prevent overfitting
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') is not None and logs.get('accuracy') > 0.95:
            print("\nReached 95 percent so cancelling training")
            self.model.stop_training = True


callbacks = myCallback()

In [11]:
# This function will be used to create our model
def create_model():
    model = keras.Sequential([

        tf.keras.layers.Dense(128, activation='relu', input_shape=[num_parameters]),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.BatchNormalization(),

    ])

    model.compile(
        #loss='sparse_categorical_crossentropy', TODO: change loss function for regression
        optimizer='adam',
        metrics=['accuracy']
    )

    return model

In [13]:
model = create_model()

In [ ]:
# This will allow us to save model weights after training
model.save_weights('./checkpoints/model_weights')